<a href="https://colab.research.google.com/github/ZeeshanAhmed95/Data-Extraction-and-Sentiment-Analysis/blob/main/Zeeshan_Blackcoffer_Projectipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing input file
df_project=pd.read_excel('/content/drive/MyDrive/Input.xlsx')[['URL_ID','URL']]

In [ ]:
#dropping a column
df_project.drop('URL_ID',axis=1,inplace=True)

In [ ]:
#viewing data in dataframe
df_project['URL'][0]

'https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/'

# Data extraction

In [ ]:
#performing data extraction on a single URL
j=df_project['URL'][40]
   
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}#giving user access
page=requests.get(j,headers=headers)#loading text in url
soup=BeautifulSoup(page.content,'html.parser')#parsing url text
content=soup.findAll(attrs={'class':'td-post-content'})#extracting only text part
content=content[0].text.replace('<p>'," ").replace('\n'," ").replace('\xa0',' ')#replace end line symbol with space 
title=soup.findAll(attrs={'class':'entry-title'})#extracting title of website
print(title)

[<h3 class="entry-title td-module-title"><a href="https://insights.blackcoffer.com/ranking-customer-behaviours-for-business-strategy/" rel="bookmark" title="Ranking customer behaviours for business strategy">Ranking customer behaviours for business strategy</a></h3>, <h3 class="entry-title td-module-title"><a href="https://insights.blackcoffer.com/algorithmic-trading-for-multiple-commodities-markets-like-forex-metals-energy-etc/" rel="bookmark" title="Algorithmic trading for multiple commodities markets, like Forex, Metals, Energy, etc.">Algorithmic trading for multiple commodities markets, like Forex, Metals, Energy, etc.</a></h3>, <h3 class="entry-title td-module-title"><a href="https://insights.blackcoffer.com/trading-bot-for-forex/" rel="bookmark" title="Trading Bot for FOREX">Trading Bot for FOREX</a></h3>, <h3 class="entry-title td-module-title"><a href="https://insights.blackcoffer.com/python-model-for-the-analysis-of-sector-specific-stock-etfs-for-investment-purposes%ef%bf%bc/"

In [ ]:
#extracting text from all the url
main_df = pd.DataFrame()
for i in range(len(df_project)):
  #  print(i) 
   j=df_project['URL'][i]
   
   headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}#giving user access
   page=requests.get(j,headers=headers)#loading text in url
   soup=BeautifulSoup(page.content,'html.parser')#parsing url text
   content=soup.findAll(attrs={'class':'td-post-content'})#extracting only text part
   try:  
    content=content[0].text.replace('<p>'," ").replace('\n'," ").replace('\xa0',' ')#replace end line symbol with space 
    title=soup.findAll(attrs={'class':'entry-title'})#extracting title of website

    title=title[0].text.replace('\n',"  ").replace('/',"")
    text=title+ '.' +content#merging title and content text
    df1=pd.DataFrame(pd.Series(text),columns=['data'])#creating series data frame
    main_df = pd.concat([df1,main_df],ignore_index=True)
   
   except:
     print(i)
     continue


   

7
20
25
107


In [ ]:
#visualizing shape of the dataframe 
main_df.shape

(110, 1)

# Data analysis

In [ ]:
#information of data frame
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    110 non-null    object
dtypes: object(1)
memory usage: 1008.0+ bytes


In [ ]:
main_df = main_df['data'].str.replace("Ranking customer behaviours for business strategy.","")

<ipython-input-13-026d50e3ba9b>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  main_df = main_df['data'].str.replace("Ranking customer behaviours for business strategy.","")


In [ ]:
main_df=pd.DataFrame(main_df,columns=['data'])

In [ ]:
main_df['data']

0       Big Data To begin with I shall first like to ...
1       Analytics is a statistical scientific process...
2       Quality and affordable healthcare is a vision...
3       What Is an Investment? An investment is a res...
4       Reconciling with the financial realities of a...
                             ...                        
105     “Machine intelligence is the last invention t...
106     “Anything that could give rise to smarter-tha...
107     Introduction AI is rapidly evolving in the em...
108     Human minds, a fascination in itself carrying...
109     Introduction “If anything kills over 10 milli...
Name: data, Length: 110, dtype: object

In [ ]:
#creating new dataframe
df = pd.DataFrame()

In [ ]:
#converting text to sentence
import re
#splitting text on '.'
df['data']= main_df['data'].apply(lambda x: x.split('([\.]\s)'))


In [ ]:
df.iloc[0]

data    [ Big Data To begin with I shall first like to...
Name: 0, dtype: object

In [ ]:
#removing . char from each rows
def abcd(x):    
    nopunc =[char for char in x if char != '.']
    return ''.join(nopunc)
df['data']=df['data'].apply(abcd)


In [ ]:
#replacing empty space with null values
df=df.replace('',np.nan,regex=True)
df=df.mask(df==" ")
df=df.dropna()
df.reset_index(drop=True,inplace=True)

In [ ]:
#importing nltk library and stopwords
import nltk
import string


In [ ]:
punc=[punc for punc in string.punctuation]

In [ ]:
#importing stop words files that are provided
SW_Auditor=pd.read_csv("/content/drive/MyDrive/StopWords_Auditor.txt",header=None)
SW_Currencies=pd.read_csv("/content/drive/MyDrive/StopWords_Currencies.txt",header=None,encoding="ISO-8859-1",sep='\n' )
SW_DatesandNumbers=pd.read_csv("/content/drive/MyDrive/StopWords_DatesandNumbers.txt",header=None)
SW_Generic=pd.read_csv("/content/drive/MyDrive/StopWords_Generic.txt",header=None)
SW_GenericLong=pd.read_csv("/content/drive/MyDrive/StopWords_GenericLong.txt",header=None)
SW_Geographic=pd.read_csv("/content/drive/MyDrive/StopWords_Geographic.txt",header=None)
SW_Names=pd.read_csv("/content/drive/MyDrive/StopWords_Names.txt",header=None)


In [ ]:
#creating func for removing stop words
def text_process(text):
    nopunc =[char for char in text if char not in punc or char not in [':',',','(',')','’','?']]
    nopunc=''.join(nopunc)
    txt=' '.join([word for word in nopunc.split() if word.lower() not in SW_Auditor])
    txt1=' '.join([word for word in txt.split() if word.lower() not in SW_Currencies])
    txt2=' '.join([word for word in txt1.split() if word.lower() not in SW_DatesandNumbers])
    txt3=' '.join([word for word in txt2.split() if word.lower() not in SW_Generic])
    txt4=' '.join([word for word in txt3.split() if word.lower() not in SW_GenericLong])
    txt5=' '.join([word for word in txt4.split() if word.lower() not in SW_Geographic])
    return ' '.join([word for word in txt5.split() if word.lower() not in SW_Names])


In [ ]:
#applying func for each row
df['data']=df['data'].apply(text_process)

In [ ]:
#importing master Dictionary
positive=pd.read_csv("/content/drive/MyDrive/positive-words.txt",header=None)
negative=pd.read_csv("/content/drive/MyDrive/negative-words.txt",header=None,encoding="ISO-8859-1",sep='\n' )


In [ ]:
positive.columns=['pos']
negative.columns=['neg']
positive['pos']=positive['pos'].astype(str)
negative['neg']=negative['neg'].astype(str)

In [ ]:
#positive and negative dictionary without stopwords
positive['pos']=positive['pos'].apply(text_process)
negative['neg']=negative['neg'].apply(text_process)

In [ ]:
#positive list
length=positive.shape[0]
pos=[]
for i in range(0,length):
   nopunc =[char for char in positive.iloc[i] if char not in string.punctuation or char != '+']
   nopunc=''.join(nopunc)

   pos.append(nopunc)

In [ ]:
#negative list
length=negative.shape[0]
neg=[]
for i in range(0,length):
  nopunc =[char for char in negative.iloc[i] if char not in string.punctuation or char != '+']
  nopunc=''.join(nopunc)
  neg.append(nopunc)

# TOKENIZATION

In [ ]:
#importing tokenize library
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row['data']), axis=1)

In [ ]:
df['tokenized_sents'].head()

0    [Big, Data, To, begin, with, I, shall, first, ...
1    [Analytics, is, a, statistical, scientific, pr...
2    [Quality, and, affordable, healthcare, is, a, ...
3    [What, Is, an, Investment, An, investment, is,...
4    [Reconciling, with, the, financial, realities,...
Name: tokenized_sents, dtype: object

In [ ]:
df['total_len'] = df['tokenized_sents'].map(lambda x: len(x))

# CALCULATING SCORES

### 1) POSITIVE SCORE

In [ ]:
num_pos = df['tokenized_sents'].map(lambda x: len([i for i in x if i in pos]))
df['positive_score'] = num_pos


### 2) NEGATIVE SCORE

In [ ]:
num_neg = df['tokenized_sents'].map(lambda x: len([i for i in x if i in neg]))
df['negative_score'] = num_neg

### 3) POLARITY SCORE

In [ ]:
#Polarity Score formula = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
Polarity_Score=(num_pos-num_neg)/((num_pos+num_neg)+0.000001)
df['Polarity_Score'] = Polarity_Score

### 4) SUBJECTIVITY SCORE

In [ ]:
#Subjectivity Score formula = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
subjectiivity_score=(num_pos-num_neg)/((len(df['tokenized_sents']))+ 0.000001)
df['subjectiivity_score'] = subjectiivity_score

### 5) AVG SENTENCE LENGTH

In [ ]:
sum_of_length_of_sents = sum(df['total_len'].values)

In [ ]:
sum_of_length_of_sents

136573

In [ ]:
length_of_sentence = []
for i in df['total_len']:
  length_of_sentence.append(i)

In [ ]:
avg_sents_length=sum_of_length_of_sents / length_of_sentence
df['avg_senetence_length'] = avg_sents_length

### 6) PERCENTAGE OF COMPLEX WORDS

In [ ]:
#complex word individual count
vowels=['a','e','i','o','u']
import re
count=0
complex_Word_Count=0
for i in str(df['tokenized_sents']):
  x=re.compile('[es|ed]$')
  if x.match(i.lower()):
   count+=0
  else:
    for j in str(df['tokenized_sents']):
      if(j.lower() in vowels ):
        count+=1
  if(count>2):
   complex_Word_Count+=1
  count=0


In [ ]:
Percentage_of_Complex_words=complex_Word_Count/df['total_len']
df['Percentage_of_Complex_words'] = Percentage_of_Complex_words

### 7) FOG INDEX

In [ ]:
#Fog Index formula = 0.4 * (Average Sentence Length + Percentage of Complex words)
Fog_Index = 0.4 * (avg_sents_length + Percentage_of_Complex_words)
df['Fog_Index'] = Fog_Index

### 8) AVG NUMBER OF WORDS PER SENTENCE

In [ ]:

avg_no_of_words_per_sentence=df['total_len']/ df.shape[0]
df['avg_no_of_words_per_sentence'] = avg_no_of_words_per_sentence

### 9) COMPLEX WORD COUNT

In [ ]:
def count_complex_words(word):  
  vowels=['a','e','i','o','u']
  import re
  count=0
  complex_Word_Count=0
  for i in word:
    x=re.compile('[es|ed]$')
    if x.match(i.lower()):
     count+=0
    else:
      for j in i:
        if(j.lower() in vowels ):
          count+=1
    if(count>2):
     complex_Word_Count+=1
    count=0
  return complex_Word_Count

In [ ]:
df['complex_Word_Count'] = df['tokenized_sents'].apply(lambda x: count_complex_words((x)))

In [ ]:
df.head()

,data,tokenized_sents,total_len,positive_score,negative_score,Polarity_Score,subjectiivity_score,avg_senetence_length,Percentage_of_Complex_words,Fog_Index,avg_no_of_words_per_sentence,complex_Word_Count
0,Big Data To begin with I shall first like to e...,"[Big, Data, To, begin, with, I, shall, first, ...",1111,45,39,0.071429,0.054545,122.927993,0.549955,49.391179,10.100000,276
1,Analytics is a statistical scientific process ...,"[Analytics, is, a, statistical, scientific, pr...",759,33,4,0.783784,0.263636,179.938076,0.805007,72.297233,6.900000,256
2,Quality and affordable healthcare is a vision ...,"[Quality, and, affordable, healthcare, is, a, ...",1247,30,44,-0.189189,-0.127273,109.521251,0.489976,44.004491,11.336364,363
3,What Is an Investment An investment is a resou...,"[What, Is, an, Investment, An, investment, is,...",1660,47,12,0.593220,0.318182,82.272892,0.368072,33.056386,15.090909,451
4,Reconciling with the financial realities of an...,"[Reconciling, with, the, financial, realities,...",976,29,28,0.017544,0.009091,139.931352,0.626025,56.222951,8.872727,262


### 10) WORD COUNT

In [ ]:
df['word_count'] = df['tokenized_sents'].map(lambda x: len(x))

### 11) SYLLABLE PER WORD

In [ ]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [ ]:
df['syllable_count'] = df['tokenized_sents'].apply(lambda x: syllable_count(str(x)))

### 12) PERSONAL PRONOUNS

In [ ]:
def pronoun_count(word):  
  pronouns=['i','we','my','ours','us' ]
  import re
  count=0
  for i in word:
    if i.lower() in pronouns:
      count+=1
  return count

In [ ]:
df['personal_pronouns'] = df['tokenized_sents'].apply(lambda x: pronoun_count((x)))

In [ ]:
df.head()

,data,tokenized_sents,total_len,positive_score,negative_score,Polarity_Score,subjectiivity_score,avg_senetence_length,Percentage_of_Complex_words,Fog_Index,avg_no_of_words_per_sentence,complex_Word_Count,word_count,syllable_count,personal_pronouns,avg_word_length
0,Big Data To begin with I shall first like to e...,"[Big, Data, To, begin, with, I, shall, first, ...",1111,45,39,0.071429,0.054545,122.927993,0.549955,49.391179,10.100000,276,1111,1942,8,4.801980
1,Analytics is a statistical scientific process ...,"[Analytics, is, a, statistical, scientific, pr...",759,33,4,0.783784,0.263636,179.938076,0.805007,72.297233,6.900000,256,759,1461,0,5.440053
2,Quality and affordable healthcare is a vision ...,"[Quality, and, affordable, healthcare, is, a, ...",1247,30,44,-0.189189,-0.127273,109.521251,0.489976,44.004491,11.336364,363,1247,2147,2,4.801925
3,What Is an Investment An investment is a resou...,"[What, Is, an, Investment, An, investment, is,...",1660,47,12,0.593220,0.318182,82.272892,0.368072,33.056386,15.090909,451,1660,2778,18,4.894578
4,Reconciling with the financial realities of an...,"[Reconciling, with, the, financial, realities,...",976,29,28,0.017544,0.009091,139.931352,0.626025,56.222951,8.872727,262,976,1697,9,5.100410


### 13) AVG WORD LENGTH

In [ ]:
def word_count_number(sentence):
  count=0
  for i in (sentence):
    for j in i:
      for k in j:
        count += 1
  return count

In [ ]:
df['number_of_words'] = df['tokenized_sents'].apply(lambda x: word_count_number((x)))

In [ ]:
df['avg_word_length'] = df['number_of_words']/df['word_count']

In [ ]:
df.drop(columns = (['number_of_words']), inplace = True)

In [ ]:
output=df.copy()

In [ ]:
output.head()

,data,tokenized_sents,total_len,positive_score,negative_score,Polarity_Score,subjectiivity_score,avg_senetence_length,Percentage_of_Complex_words,Fog_Index,avg_no_of_words_per_sentence,complex_Word_Count,word_count,syllable_count,personal_pronouns,avg_word_length
0,Big Data To begin with I shall first like to e...,"[Big, Data, To, begin, with, I, shall, first, ...",1111,45,39,0.071429,0.054545,122.927993,0.549955,49.391179,10.100000,276,1111,1942,8,4.801980
1,Analytics is a statistical scientific process ...,"[Analytics, is, a, statistical, scientific, pr...",759,33,4,0.783784,0.263636,179.938076,0.805007,72.297233,6.900000,256,759,1461,0,5.440053
2,Quality and affordable healthcare is a vision ...,"[Quality, and, affordable, healthcare, is, a, ...",1247,30,44,-0.189189,-0.127273,109.521251,0.489976,44.004491,11.336364,363,1247,2147,2,4.801925
3,What Is an Investment An investment is a resou...,"[What, Is, an, Investment, An, investment, is,...",1660,47,12,0.593220,0.318182,82.272892,0.368072,33.056386,15.090909,451,1660,2778,18,4.894578
4,Reconciling with the financial realities of an...,"[Reconciling, with, the, financial, realities,...",976,29,28,0.017544,0.009091,139.931352,0.626025,56.222951,8.872727,262,976,1697,9,5.100410
